# Model Procedure

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1) Train part for the model
## It's a necessary part for using these pre-trained model to different spieces

In this part, you can train model to match your dataset.<br /> For example, the pre-trained model is based on the human brain, if you want to use it on mice brains,
 you should train it using your own dataset

### Part 1: import necessary packages

In [2]:
!pip install nilearn
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.6 MB 21.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52.8 MB 245 kB/s 


In [23]:
import os, sys, pickle
sys.path.append('/content/drive/MyDrive/UNet_Model')
from arguments import Args, Args_test, Args_baco
from orientation import orientation
from bias_correction import batching_baco
from Train import Train
from Test import Test
import numpy as np
import nilearn as nil
import nibabel as nib
from nilearn import plotting
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

### Part 2: set the parameters we need

#### The followings are the parameters we can set:
|Parameters|Meaning|Default value|Whether the input is required|
| :----: | :------: | :----: | :----:|
|train_t1w|Train T1w Directory|None|Yes|
|train_msk|Train Mask Directory|None|Yes|
|out_dir|Output Directory|None|Yes|
|validate_t1w|Validation T1w Directory|None|Yes|
|validate_msk|Validation Mask Directory|None|Yes|
|init_model|Initial Model Directory or Filename|None|Yes|
|conv_block|Num of UNet Blocks|5|Optional|
|input_slice|Num of Slices for Model Input|3|Optional|
|kernel_root|Num of the Root of Kernel|16|Optional|
|rescale_dim|Dimension to Rescale|256|Optional|
|num_epoch|Num of Epoch|40|Optional|
|learning_rate|Learning Rate of the Model|0.0001|Optional|
|optimizerSs|Optimizer|Adam|Can't be set|

#### Also, we can change some parameters individually
|Parameters|Function to Change it|
| :----: | :------: |
|train_t1w|change_train_t1w|
|train_msk|change_train_msk|
|out_dir|change_out_dir|
|validate_t1w|change_validate_t1w|
|validate_msk|change_validate_msk|
|init_model|change_init_model|

### Here is a brief introduction for each pretrained model we provide
1. **Site-All-T-epoch_36.model**: Trained on 12 macaques across 6 sites (2 macaques per site) from PRIME-DE. Six sites include newcastle, ucdavis, oxford, ion, ecnu-chen, and sbri.

2. **Site-All-T-epoch_36_update_with_Site_6_plus_7-epoch_09.model**: Trained on 19 macaques across 13 sites from PRIME-DE (12 macaques across 6 sites used in the first model and 7 macaques across 7 additional sites) Seven sites include NIMH, ecnu-k, nin, rockefeller, uwo, mountsinai-S, and lyon.

3. **Site-All-T-epoch_36_update_with_Site_*.model**: Site-specific model for NIMH, ecnu-k, nin, rockefeller, uwo, mountsinai-S, and lyon.

4. **Site-All-T-epoch_36_update_with_Site_Pigs_09.model**: The **pig** model - Trained on 12 macaques and updated with the pig data (N=3).

#### [Download the models](https://github.com/HumanBrainED/NHP-BrainExtraction/tree/master/UNet_Model)

In [ ]:
args = Args(train_t1w="/content/drive/MyDrive/Train_3/Train_data"
           , train_msk="/content/drive/MyDrive/Train_Val_Test_model_3/Train_data.zip (Unzipped Files)/Train_msk_2"
           , out_dir="/content/drive/MyDrive/Train_Val_Test_model_3/Train_data.zip (Unzipped Files)/Out_Dir/model_noba_model"
           , validate_t1w="/content/drive/MyDrive/Train_3/Val_data"
           , validate_msk="/content/drive/MyDrive/Train_Val_Test_model_3/Train_data.zip (Unzipped Files)/Val_msk_2"
           , init_model="/content/drive/MyDrive/UNet_Model/models/Site-Human-epoch_08.model"
           , num_epoch=50
           , rotation=True)

### Part 3: Check the Orientation & Resample the Image

In [ ]:
ort = orientation(t1_path = args.train_t1w)
if ort.check() == False:
    args.train_t1w = ort.orient()

ort.path = args.train_msk
if ort.check() == False:
    args.train_msk = ort.orient(mode='mask')

ort.path = args.validate_t1w
if ort.check() == False:
    args.validate_t1w = ort.orient()
    
ort.path = args.validate_msk
if ort.check() == False:
    args.validate_msk = ort.orient(mode='mask')

image : 4_6_d_163930.nii
original shape: (256, 256, 30) resample shape: (256, 108, 256)
image : 4_12_d_183819.nii
original shape: (256, 256, 23) resample shape: (256, 82, 256)
image : 1_1_d.nii
original shape: (256, 256, 30) resample shape: (256, 187, 256)
image : 2_1_d.nii
original shape: (256, 256, 30) resample shape: (256, 187, 256)
image : 3_d (1).nii
original shape: (360, 320, 36) resample shape: (360, 351, 360)
image : 3_d (4).nii
original shape: (360, 320, 36) resample shape: (360, 351, 360)
mask : 4_12_m_183819.nii.gz
original shape: (256, 256, 34) resample shape: (256, 122, 257)
mask : 4_6_m_163930.nii.gz
original shape: (256, 256, 30) resample shape: (256, 108, 257)
mask : 1_1_m.nii
original shape: (256, 256, 30) resample shape: (256, 187, 257)
mask : 2_1_m.nii
original shape: (256, 256, 30) resample shape: (256, 187, 257)
mask : 3_m (1).nii.gz
original shape: (360, 320, 36) resample shape: (360, 352, 360)
mask : 3_m (4).nii


### Part 4: Bias Correction

In [ ]:
args_baco_train = Args_baco(input=args.train_t1w)
# args_baco_val = Args_baco(input=args.validate_t1w)

args.train_t1w = batching_baco(args_baco_train)
# args.val_t1w = batching_baco(args_baco_val)

### Part 5: Train the Model & Save it

In [ ]:
model_train = Train(args=args)
Dict_DL = model_train.train()

## 2) Test part for the model

### *Notice*: Here we need to reset the parameters

### Part 1: Reset the parameters
|Parameters|Meaning|Default value|Whether the input is required|
| :----: | :------: | :----: | :----:|
|test_t1w|Test T1w Directory|None|Yes|
|test_msk|Test Mask Directory|None|Yes|
|out_dir|Output Directory|None|Yes|
|test_model|Test Model Directory or Filename|None|Yes|
|conv_block|Num of UNet Blocks|5|Optional|
|input_slice|Num of Slices for Model Input|3|Optional|
|kernel_root|Num of the Root of Kernel|16|Optional|
|rescale_dim|Dimension to Rescale|256|Optional|

#### Some parameters that can be changed individually
|Parameters|Function to Change it|
| :----: | :------: |
|test_t1w|change_test_t1w|
|test_msk|change_test_msk|
|out_dir|change_out_dir|
|test_model|change_test_model|

In [19]:
args_test = Args_test(test_t1w="/content/drive/MyDrive/course_bme/image"
           , test_msk="/content/drive/MyDrive/course_bme/Mask"
           , out_dir="/content/drive/MyDrive/course_bme/model_out"
           , test_model="/content/drive/MyDrive/course_bme/model-49-epoch.model")

### Part 2: Check the Orientation & Resample the Image

In [20]:
ort_test = orientation(t1_path = args_test.test_t1w)
if ort_test.check() == False:
    args_test.test_t1w = ort_test.orient()

ort_test.path = args_test.test_msk
if ort_test.check() == False:
    args_test.test_msk = ort_test.orient(mode='mask')

image : 1_7_d.nii
original shape: (256, 256, 30) resample shape: (256, 187, 256)
mask : 1_7_m.nii
original shape: (256, 256, 30) resample shape: (256, 187, 257)


### Part 3: Bias Correction

In [24]:
args_baco_test = Args_baco(input=args_test.test_t1w)

args_test.test_t1w = batching_baco(args_baco_test)

Processing new_1_7_d.nii


### Part 4: Test the Model

In [39]:
model_test = Test(args=args_test)
dice_dict = model_test.test()

===================================Testing Model====================================
baco_new_1_7_d.nii
1_7_m.nii


/usr/local/lib/python3.7/dist-packages/nilearn/image/resampling.py:453: UserWarning: The provided image has no sform in its header. Please check the provided file. Results may not be as expected.
  warnings.warn("The provided image has no sform in its header. "


	0.9654 +/- 0.0000
